In [7]:
import numpy as np
data = np.load('/tmp/game_data.npy')
labels = np.load('/tmp/game_labels.npy')



In [8]:
#Filter out the end state from data. 
# This will be a matrix with the game state afer x seconds in each column
# the first row is time
# following by gold, last hits and xp for each player
end_5_min_games = np.array([game[2:,-1] for game in data]).T
#print the data to look at it
print('Example data table. First 10 matches. Game state at the end of 5 min')
print(end_5_min_games[:,:10])
# the shape should be 31, 50000. 
print('The shape of the match data table, first row is the time column, which is always 300')
print(end_5_min_games.shape)

Example data table. First 10 matches. Game state at the end of 5 min
[[1056  798 1140 ..., 1063 1797  836]
 [   1    0   12 ...,   16   27    8]
 [ 649  910 1291 ...,  834 1504 1105]
 ..., 
 [   0    0    0 ...,    0    0    0]
 [   0    0    0 ...,    0    0    0]
 [   0    0    0 ...,    0    0    0]]
The shape of the match data table, first row is the time column, which is always 300
(2210, 50000)


In [9]:
from keras.layers import Dense, Activation
from keras.models import Sequential
import keras

model = Sequential()

model.add(Dense(units=50, input_dim=2210))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.0, nesterov=False),
             metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])


Using TensorFlow backend.


In [10]:
print('Shapes of labels')
print(labels.shape)
from keras.utils.np_utils import to_categorical
labels_binary = to_categorical(labels)
print('example of how the labels look like')
print(labels[:10])
print('Re-structure the labels so that it is two dimensions, with prob value of each team winning')
print(labels_binary.shape)
print('example of how the binary labels look like')
print(labels_binary[:10])

# separate into train and test
x_train = end_5_min_games[:,:40000]
y_train = labels_binary[:40000,:]
x_test = end_5_min_games[:,40000:]
y_test = labels_binary[40000:,:]
print(y_test.shape)


Shapes of labels
(50000, 1)
example of how the labels look like
[[1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Re-structure the labels so that it is two dimensions, with prob value of each team winning
(50000, 2)
example of how the binary labels look like
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
(10000, 2)


In [11]:
res = model.fit(x_train.T, y_train, epochs=50, batch_size=1000, validation_split=0.8)

Train on 7999 samples, validate on 32001 samples
Epoch 1/50
7999/7999 [==============================] - 0s - loss: 7.1006 - mean_absolute_error: 0.4705 - categorical_accuracy: 0.5294 - val_loss: 7.1112 - val_mean_absolute_error: 0.4697 - val_categorical_accuracy: 0.5303
Epoch 2/50
7999/7999 [==============================] - 0s - loss: 7.0095 - mean_absolute_error: 0.4715 - categorical_accuracy: 0.5278 - val_loss: 6.8251 - val_mean_absolute_error: 0.4602 - val_categorical_accuracy: 0.5396
Epoch 3/50
7999/7999 [==============================] - 0s - loss: 6.7497 - mean_absolute_error: 0.4580 - categorical_accuracy: 0.5416 - val_loss: 6.5793 - val_mean_absolute_error: 0.4513 - val_categorical_accuracy: 0.5494
Epoch 4/50
7999/7999 [==============================] - 0s - loss: 6.6801 - mean_absolute_error: 0.4561 - categorical_accuracy: 0.5442 - val_loss: 7.3709 - val_mean_absolute_error: 0.4703 - val_categorical_accuracy: 0.5298
Epoch 5/50
7999/7999 [==============================] - 0s 

7999/7999 [==============================] - 0s - loss: 5.9105 - mean_absolute_error: 0.4064 - categorical_accuracy: 0.5928 - val_loss: 6.0095 - val_mean_absolute_error: 0.4121 - val_categorical_accuracy: 0.5878
Epoch 38/50
7999/7999 [==============================] - 0s - loss: 6.4502 - mean_absolute_error: 0.4323 - categorical_accuracy: 0.5681 - val_loss: 6.2792 - val_mean_absolute_error: 0.4212 - val_categorical_accuracy: 0.5786
Epoch 39/50
7999/7999 [==============================] - 0s - loss: 6.1636 - mean_absolute_error: 0.4155 - categorical_accuracy: 0.5843 - val_loss: 6.0004 - val_mean_absolute_error: 0.4086 - val_categorical_accuracy: 0.5915
Epoch 40/50
7999/7999 [==============================] - 0s - loss: 6.0472 - mean_absolute_error: 0.4129 - categorical_accuracy: 0.5868 - val_loss: 5.7006 - val_mean_absolute_error: 0.3976 - val_categorical_accuracy: 0.6025
Epoch 41/50
7999/7999 [==============================] - 0s - loss: 6.1195 - mean_absolute_error: 0.4178 - categoric

In [14]:
# Test loss and accuracy
loss_and_metrics = model.evaluate(x_test.T, y_test)
print(loss_and_metrics)

10000/10000 [==============================] - 0s     
[5.863969346618652, 0.4006606939315796, 0.59960000000000002]
